### Mounting drive on Colab

In [1]:
from google.colab import drive
drive.mount('/gdrive', force_remount = True)

Mounted at /gdrive


###Libraries

In [2]:
!pip install tensorflow_addons

     |████████████████████████████████| 1.1 MB 12.1 MB/s 


In [3]:
import os
import cv2
import os
import h5py
import numpy as np
import pickle
from tqdm import tqdm
from PIL import Image
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow_addons.metrics import HammingLoss
from tensorflow_addons.metrics import F1Score
os.chdir('/gdrive/My Drive/')
from numpy import savez_compressed
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras import layers, models
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model

## Effect of changing dropout rate

To understand the effects of dropouts two experimental setups were created:

1. The probability of the dropout in a hidden layer is varied from 0.1 to 0.9
2. The number of neurons are adjusted according the dropout rate in each layer to compensate the loss of neurons for each dropout rate.


        
      Let the structure of hidden layers in ResNet50 be: [2000, 1000, 500]  
      
      1st Experimental setup: 
      
      dropout rate i.e 'p' is varied from 0.0 to 0.9 with a stride of 0.1
      The number of neurons in each layer i.e 'n' is maintained constant so the number of neurons after dropout reduce progrssively.

      2nd Experimental setup:

      Dropout rate i.e 'p' is varied from 0.0 to 0.9 with a stride of 0.1
      The number of neurons i.e 'n' are adjusted such tht 'p*n' remains constant for each dropout rate.

      For Ex: if p = 0.1
              and n = 2000

              n after dropout = 1800

              for number of neurons to remain constant i.e 2000 after dropout 
              
              neurons are adjusted to 2222 such that 'n' after dropout is still 2000.



### For Imagenet as a source "pxn" remains constant

In [7]:
dropout_rate = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [8]:
layer_size = {}
layer = 1
for i in [2000, 1000, 500]:
  lis = []
  for j in reversed(range(1, 10)):
    lis.append((i*10)//j)
    layer_size[layer] = lis
  layer+=1

print(layer_size)

{1: [2222, 2500, 2857, 3333, 4000, 5000, 6666, 10000, 20000], 2: [1111, 1250, 1428, 1666, 2000, 2500, 3333, 5000, 10000], 3: [555, 625, 714, 833, 1000, 1250, 1666, 2500, 5000]}


In [9]:
from tensorflow.keras.applications.resnet50 import ResNet50
from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()

pre_trained_model = ResNet50(input_shape = (96, 96, 3), 
                                include_top = False, 
                                weights = None)
#for layer in pre_trained_model.layers:
#  layer.trainable = False

In [10]:
def compile_model(num_neurons, dropout_rate):
  from tensorflow.keras.optimizers import RMSprop

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(pre_trained_model.output)
  
  for i in range(3):
    # Add a fully connected layer with "num_neurons" hidden units and ReLU activation
    x = layers.Dense(num_neurons[i], activation='relu')(x)                  
    x = Dropout(dropout_rate)(x)
  # Add a final sigmoid layer for classification
  x = layers.Dense  (15, activation='softmax')(x)           

  model = Model( pre_trained_model.input, x) 

  model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc', tf.keras.metrics.AUC(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), HammingLoss(threshold=0.5, mode='multiclass'), F1Score(num_classes= 15, threshold=0.5)])
  return model

In [11]:
lis= []
model = []
rate = 0.1
for i in range(0, 9):
  lis.append(layer_size[1][i])
  lis.append(layer_size[2][i])
  lis.append(layer_size[3][i])
  model.append(compile_model(lis, rate))
  rate+=0.1

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


In [12]:
data = np.load('Xtrain.npy', allow_pickle=True)
labels = np.load('ytrain.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42, stratify = labels)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42, stratify = y_test)

In [13]:
y_train = to_categorical(y_train, 15)
y_val = to_categorical(y_val, 15)
y_test = to_categorical(y_test, 15)

In [14]:
for imagenet_resnet50 in model:
  imagenet_resnet50.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
  test_acc = imagenet_resnet50.evaluate(X_test, y_test)
  print("***********test accuracy is", test_acc,"***************")
  print("***********train time is", sum(cb.logs),"***************")
  print("f1 score is", sum(test_acc[6])/15)

Epoch 1/10
200/200 [==============================] - 165s 690ms/step - loss: 3.1046 - acc: 0.0722 - auc: 0.5180 - recall: 0.0048 - precision: 0.1005 - hamming_loss: 0.9952 - f1_score: 0.0090 - val_loss: 2.7151 - val_acc: 0.0640 - val_auc: 0.4996 - val_recall: 0.0000e+00 - val_precision: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
200/200 [==============================] - 136s 680ms/step - loss: 2.6314 - acc: 0.1068 - auc: 0.6178 - recall: 0.0026 - precision: 0.2121 - hamming_loss: 0.9974 - f1_score: 0.0051 - val_loss: 2.5868 - val_acc: 0.1317 - val_auc: 0.6612 - val_recall: 0.0063 - val_precision: 0.4318 - val_hamming_loss: 0.9937 - val_f1_score: 0.0111
Epoch 3/10
200/200 [==============================] - 136s 680ms/step - loss: 1.9855 - acc: 0.3008 - auc: 0.8406 - recall: 0.0892 - precision: 0.5780 - hamming_loss: 0.9108 - f1_score: 0.1391 - val_loss: 1.9535 - val_acc: 0.3330 - val_auc: 0.8478 - val_recall: 0.1323 - val_precision: 0.5387 - val_hammin

### For image net as a source "n" remains constant

In [15]:
dropout_rate = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

In [16]:
from tensorflow.keras.applications.resnet50 import ResNet50
from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

cb = TimingCallback()

pre_trained_model = ResNet50(input_shape = (96, 96, 3), 
                                include_top = False, 
                                weights = None)
for layer in pre_trained_model.layers:
  layer.trainable = False

In [17]:
def compile_model(dropout_rate):
  from tensorflow.keras.optimizers import RMSprop

  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(pre_trained_model.output)
  
  # Add a fully connected layer with "num_neurons" hidden units and ReLU activation
  x = layers.Dense(2000, activation='relu')(x)                  
  x = Dropout(dropout_rate)(x)

  x = layers.Dense(1000, activation='relu')(x)                  
  x = Dropout(dropout_rate)(x)

  x = layers.Dense(500, activation='relu')(x)                  
  x = Dropout(dropout_rate)(x)

  # Add a final sigmoid layer for classification
  x = layers.Dense  (15, activation='softmax')(x)           

  model = Model( pre_trained_model.input, x) 

  model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'categorical_crossentropy', 
              metrics = ['acc', tf.keras.metrics.AUC(), tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), HammingLoss(threshold=0.5, mode='multiclass'), F1Score(num_classes= 15, threshold=0.5)])
  return model

In [18]:
data = np.load('Xtrain.npy', allow_pickle=True)
labels = np.load('ytrain.npy', allow_pickle=True)

X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2, random_state = 42, stratify = labels)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size = 0.5, random_state = 42, stratify = y_test)

In [19]:
y_train = to_categorical(y_train, 15)
y_val = to_categorical(y_val, 15)
y_test = to_categorical(y_test, 15)

In [20]:
for rate in dropout_rate:
  imagenet_resnet50 = compile_model(rate)
  imagenet_resnet50.fit(X_train, y_train, batch_size = 120, epochs=10, validation_data=(X_val, y_val), callbacks = [cb])
  test_acc = imagenet_resnet50.evaluate(X_test, y_test)
  print("***********test accuracy is", test_acc,"***************")
  print("***********train time is", sum(cb.logs),"*************")
  print("f1 score is", sum(test_acc[6])/15)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/rmsprop.py:130: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(RMSprop, self).__init__(name, **kwargs)


Epoch 1/10
200/200 [==============================] - 50s 230ms/step - loss: 5.9710 - acc: 0.0654 - auc_9: 0.4999 - recall_9: 0.0137 - precision_9: 0.0669 - hamming_loss: 0.9863 - f1_score: 0.0227 - val_loss: 2.7730 - val_acc: 0.0667 - val_auc_9: 0.4994 - val_recall_9: 0.0000e+00 - val_precision_9: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 2/10
200/200 [==============================] - 44s 223ms/step - loss: 2.7213 - acc: 0.0661 - auc_9: 0.4982 - recall_9: 0.0000e+00 - precision_9: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.7081 - val_acc: 0.0673 - val_auc_9: 0.5002 - val_recall_9: 0.0000e+00 - val_precision_9: 0.0000e+00 - val_hamming_loss: 1.0000 - val_f1_score: 0.0000e+00
Epoch 3/10
200/200 [==============================] - 44s 222ms/step - loss: 2.7113 - acc: 0.0670 - auc_9: 0.5007 - recall_9: 0.0000e+00 - precision_9: 0.0000e+00 - hamming_loss: 1.0000 - f1_score: 0.0000e+00 - val_loss: 2.7080 - val_acc: 0.0673 - val_auc_9:

InternalError: ignored